In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferWindowMemory
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(temperature=0.1)

memory = ConversationBufferWindowMemory(
    llm=llm,
    return_messages=True,
    k=10,
)

examples = [
    {
        "movie":"top gun",
        "answer": "🛩️👨‍✈️🔥"
    },
    {
        "movie":"the godfather",
        "answer": "👨‍👨‍👦🔫🍝"
    },
    {
        "movie":"the batman",
        "answer": "🦇🥷👊"
    },
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "Express {movie} with three emojis"),
    ("ai", "{answer}"),
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an emoji artist who can represent any object with three emojis."),
    example_prompt,
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question}) 
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [2]:
invoke_chain("interstellar")

content='🚀⏳🌌'


In [3]:
invoke_chain("lalaland")

content='🎶💃🌆'


In [4]:
invoke_chain("What are the most recent movies I asked about?")

content='The most recent movies you asked about are "Interstellar" and "La La Land."'


In [5]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='interstellar'),
  AIMessage(content='🚀⏳🌌'),
  HumanMessage(content='lalaland'),
  AIMessage(content='🎶💃🌆'),
  HumanMessage(content='What are the most recent movies I asked about?'),
  AIMessage(content='The most recent movies you asked about are "Interstellar" and "La La Land."')]}